In [1]:
from Selector import *
import sys
import time
from view.HTML import show
import json
from indicators.MA import *
from simulation.Simulator import *
from indicators.Trend import *
sys.path.append("../data")
import KDataDao as kds
from view.ViewDataFactory import *
from strategy.sequence.StrategyS1 import *
from simulation.SequenceSimulatorS import *
from strategy.sequence.MAStrategyS import *
from strategy.sequence.RStrategy import * 
from strategy.sequence.StrategyS import *
from indicators.TrendArea import *
from strategy.sequence.StrategyS1 import *
from strategy.component.buy.BuyA import *
from strategy.component.buy.BuyB import *
from strategy.component.buy.BuyC import *
from strategy.component.buy.BuyD import *
from strategy.component.buy.BuyZone import *
from strategy.component.buy.BuyBS import *
from strategy.component.sell.SellA import *
from strategy.component.sell.SellB import *
from strategy.component.sell.SellC import *
from strategy.component.sell.SellK import *
from strategy.component.sell.SellZone  import *
from strategy.component.sell.SellS import *
from strategy.component.sell.SellBS import *
from strategy.component.ConditionBean import *

In [2]:
buys = [
#     BuyA(),
#             BuyB(),
#             BuyC(),
#             BuyD(),
#             BuyZone(),
            BuyBS()
]
sells = [
#             SellA(['a','b','c','d']),
#             SellB(['a','b','c','d']),
#             SellC(['a','b','c','d']),
#             SellK(['b','c','d']),
#             SellS(['a','b','c','d']),
#             SellZone(['z']),
            SellBS(['bs'])
]         

In [3]:
class KStrategy:
    """
    K线策略
    """
    def __init__(self,trader=None,listK=None):
        self.point = 0 
        self.trend = TrendArea()
        self.condition = None
        self.listK = None
        self.trader = None
    def init(self,trader,listK=None):
        self.trader = trader
        self.listK = listK
        self.trend = TrendArea()
    def decide(self,k):
        if self.trader.isHold(k.code):
            if k.isDown():
                self.trader.sell(k.code, k.date, k.close, condition="k1")
        else:
            if k.isUp():
                self.trader.buy(k.code, k.date, k.close, condition="k1")

In [4]:
class TStrategy:
    """
    K线策略
    """
    def __init__(self,trader=None,listK=[]):
        self.point = 0 
        self.trend = TrendArea()
        self.condition = None
        self.listK = None
        self.trader = None
    def init(self,trader,listK=[]):
        self.trader = trader
        self.listK = listK
        self.trend = TrendArea()
    def decide(self,k):
        self.listK.append(k)
        if len(self.listK)<3:
            return
        pre = self.listK[-2]
        pre2 = self.listK[-3]
        if self.trader.isHold(k.code):
            if k.isDown():
                self.trader.sell(k.code, k.date, k.close, condition="k1")
        else:
            if k.isUp() and pre.isDown() and pre.close > pre2.MOC() and k.close > pre.open:
                self.trader.buy(k.code, k.date, k.close, condition="k1")

In [5]:
#测试策略
bDate = "2013-06-05"
eDate = "2016-11-07"
market = "sza"
begin = time.time()
print 'runing......'
s1 = StrategyS(buys,sells)
# s1 = TStrategy()
sim1 = SequenceSimulatorS(s1)
sim1.run(market,bDate,eDate,threadCount=0)
print 'complete %s second' %  (time.time()-begin)

runing......
thread-main begin run
thread-main end run
complete 234.008919001 second


In [6]:
# begin = time.time()
# print 'runing......'
# s12 = KStrategy2()
# sim12 = SequenceSimulator(s12)
# sim12.run(market,bDate,eDate,threadCount=0)
# print 'complete %s second' %  (time.time()-begin)

In [7]:
#均线策略
begin = time.time()
print 'runing......'
s2 = MAStrategyS(60)
sim2 = SequenceSimulatorS(s2)
sim2.run(market,bDate,eDate,threadCount=0)
print 'complete %s second' %  (time.time()-begin)

runing......
thread-main begin run
thread-main end run
complete 31.9302048683 second


In [8]:
#K线策略
begin = time.time()
print 'runing......'
ks = KStrategy()
simK = SequenceSimulatorS(ks)
simK.run(market,bDate,eDate,threadCount=0)
print 'complete %s second' %  (time.time()-begin)

runing......
thread-main begin run
thread-main end run
complete 28.8657910824 second


In [9]:
#择时收益
sim3 = Simulator(RStrategy())
sim3.runSingle('zs_000001',begin=bDate,end=eDate)

In [10]:
listK = kd.get_k_data('zs_000001',bDate,eDate)
data,volume = ViewDataFactory.getCandlestick(listK)

In [11]:
avgEarningsLine,maxCode,maxEarningsLine,minCode,minEarningsLine = sim1.earningsLines()
oddsLine,win,lose,odds = sim1.getOdds()

# avgEarningsLine2,maxCode2,maxEarningsLine2,minCode2,minEarningsLine2 = sim12.earningsLines()
maEarningsLine,maMaxCode,maMaxEarningsLine,maMinCode,maMinEarningsLine = sim2.earningsLines()
kEarningsLine,kMaxCode,kMaxEarningsLine,kMinCode,kMinEarningsLine = simK.earningsLines()

rEarningsLine = sim3.trader.earningsLine


params = {}
params["maxCode"] = maxCode
params["minCode"] = minCode
params["avgEarningsLine"] = json.dumps(avgEarningsLine)
params["avgEarningsLine2"] = {}#json.dumps(avgEarningsLine2)
params["maxEarningsLine"] = json.dumps(maxEarningsLine)
params["minEarningsLine"] = json.dumps(minEarningsLine)
params["maEarningsLine"] = json.dumps(maEarningsLine)
params["rEarningsLine"] = json.dumps(rEarningsLine)
params["kEarningsLine"] = json.dumps(kEarningsLine)
params["oddsLine"] = json.dumps(oddsLine)
params["win"] = json.dumps(win)
params["lose"] = json.dumps(lose)
params["odds"] = json.dumps(odds)
params["data"] = json.dumps(data)
params["volume"] = json.dumps(volume)
clear_output()
s = show("Strategy.html",params)
